<a href="https://colab.research.google.com/github/dattali18/IR_Assignments/blob/main/Assignment.05/IR_05_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 64.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [3]:
# Install required libraries
!pip install transformers sentence-transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 155.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-n

In [6]:
# download the files from github
sentences_url = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.05/data/sentences/sentences_calssify.csv?raw=true"

In [7]:
# download all the file into the same dir as the note book
import urllib.request

urllib.request.urlretrieve(sentences_url, 'sentences.csv')

print("Files downloaded successfully!")

Files downloaded successfully!


In [8]:
import pandas as pd

df = pd.read_csv("sentences.csv")

In [9]:
import torch
from transformers import BertTokenizer, BertModel
from sentence_transformers import SentenceTransformer

In [10]:
# Load BERT model and tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Load SBERT model
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
df.head()

,Unnamed: 0,id,sentence,pro-israel,pro-palestine,neutral,anti-israel,anti-palestine
0,0,aj_1,pope renews call for gaza ceasefire release o...,0,0,1,0,0
1,1,aj_1,Pope Francis says his thoughts go to those fac...,0,0,1,0,0
2,2,aj_1,pope francis has renewed calls for an immediat...,0,1,0,0,0
3,3,aj_2,biden is still the best us president israel co...,1,0,0,0,0
4,4,aj_2,united states president ronald reagans order t...,0,0,0,1,0


In [28]:
def get_bert_embedding(sentence):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    bert_model.to(device)
    inputs = bert_tokenizer(sentence, return_tensors='pt', truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    embeddings = outputs.last_hidden_state.sum(dim=1).squeeze().cpu().numpy()
    return embeddings

In [29]:
def get_sbert_embedding(sentence):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    return sbert_model.encode(sentence, device=device)

In [30]:
s1 = "This is a sentence"

v1 = get_bert_embedding(s1)
v2 = get_sbert_embedding(s1)
print(v1.shape)
print(v2.shape)

(768,)
(384,)


In [31]:
df["bert_embedding"] = None

for index, row in df.iterrows():
    if index % 1000 == 0:
        print(index)
    sentence = str(row["sentence"])
    bert_embedding = get_bert_embedding(sentence)
    df.at[index, "bert_embedding"] = list(bert_embedding)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000


In [35]:
# Generate SBERT embeddings
df['sbert_embedding'] = None

for index, row in df.iterrows():
  if index % 1000 == 0:
    print(index)
  sentence = str(row['sentence'])
  sbert_embedding = get_sbert_embedding(sentence)
  df.at[index, 'sbert_embedding'] = list(sbert_embedding)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000


In [36]:
# Display the updated DataFrame
df.head()

,Unnamed: 0,id,sentence,pro-israel,pro-palestine,neutral,anti-israel,anti-palestine,bert_embedding,sbert_embedding
0,0,aj_1,pope renews call for gaza ceasefire release o...,0,0,1,0,0,"[-5.398319, -3.4624884, 1.2158442, -6.873296, ...","[0.0042264466, 0.09409609, 0.02210562, 0.00452..."
1,1,aj_1,Pope Francis says his thoughts go to those fac...,0,0,1,0,0,"[-3.1675782, 4.4570203, 5.6814823, -2.0597496,...","[0.02103883, 0.060152613, 0.045743383, 0.06227..."
2,2,aj_1,pope francis has renewed calls for an immediat...,0,1,0,0,0,"[-15.144646, -0.41924834, 6.110239, -18.322815...","[0.045155767, 0.08643171, 0.058290135, -0.0015..."
3,3,aj_2,biden is still the best us president israel co...,1,0,0,0,0,"[2.12351, -1.1968501, -5.893949, 0.84273064, 0...","[0.0011824196, 0.066507965, 0.08534787, -0.064..."
4,4,aj_2,united states president ronald reagans order t...,0,0,0,1,0,"[-7.5038824, 10.2584505, -15.679925, -11.71334...","[0.07076625, 0.1025179, -0.022078907, -0.04288..."


In [38]:
# remove the unnamed col

df.columns

Index(['Unnamed: 0', 'id', 'sentence', 'pro-israel', 'pro-palestine',
       'neutral', 'anti-israel', 'anti-palestine', 'bert_embedding',
       'sbert_embedding'],
      dtype='object')

In [39]:
# delte the Unnamed col

df.drop(columns=["Unnamed: 0"], inplace=True)

In [40]:
df.head()

,id,sentence,pro-israel,pro-palestine,neutral,anti-israel,anti-palestine,bert_embedding,sbert_embedding
0,aj_1,pope renews call for gaza ceasefire release o...,0,0,1,0,0,"[-5.398319, -3.4624884, 1.2158442, -6.873296, ...","[0.0042264466, 0.09409609, 0.02210562, 0.00452..."
1,aj_1,Pope Francis says his thoughts go to those fac...,0,0,1,0,0,"[-3.1675782, 4.4570203, 5.6814823, -2.0597496,...","[0.02103883, 0.060152613, 0.045743383, 0.06227..."
2,aj_1,pope francis has renewed calls for an immediat...,0,1,0,0,0,"[-15.144646, -0.41924834, 6.110239, -18.322815...","[0.045155767, 0.08643171, 0.058290135, -0.0015..."
3,aj_2,biden is still the best us president israel co...,1,0,0,0,0,"[2.12351, -1.1968501, -5.893949, 0.84273064, 0...","[0.0011824196, 0.066507965, 0.08534787, -0.064..."
4,aj_2,united states president ronald reagans order t...,0,0,0,1,0,"[-7.5038824, 10.2584505, -15.679925, -11.71334...","[0.07076625, 0.1025179, -0.022078907, -0.04288..."


In [41]:
df.shape

(46242, 9)

In [42]:
df.to_csv("sentences_embedding.csv", index=False)

from google.colab import files

files.download("sentences_embedding.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>